In [20]:
import pandas as pd
import numpy as np
import matplotlib as plt
import geopandas as gpd
import folium
import networkx as nx
import osmnx as ox

In [105]:
#Extract subnetwork
highway_shapefile = 'Shapefiles/Snelheid_Wegvakken.shp'
network_temp = gpd.read_file(highway_shapefile)
#network_temp.plot()
#extract network A200
a200_network = network_temp.loc[np.where(network_temp['WEGNR_HMP'] == 'A200')]

df = gpd.GeoDataFrame()

df['Road_section_id'] = network_temp['WVK_ID']
df['Road_number'] = network_temp['WEGNR_HMP']
df['geometry'] = network_temp['geometry']
df_a200 = df.loc[np.where(df['Road_number'] == 'A200')]

C:\Users\syaka\AppData\Local\Temp\ipykernel_30336\857271369.py:12: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  df['geometry'] = network_temp['geometry']


In [71]:
incident = pd.read_csv('incidents19Q3Q4.csv')
incident.rename(columns = {'vild_primair_wegnummer':'Road_number', 
                           'primaire_locatie_lengtegraad':'longitude', 
                           'primaire_locatie_breedtegraad':'latitude'}, inplace=True)
incident_a200 = incident.loc[np.where(incident['Road_number'] == 'A200')]


In [28]:
Geometry = gpd.points_from_xy(incident.latitude, incident.longitude)
gdf_incident = gpd.GeoDataFrame(incident, geometry=Geometry)
gdf_incident_a200 =gdf_incident.loc[np.where(gdf_incident['Road_number'] == 'A200')]

,Unnamed: 0,id,type,starttime_new,endtime_new,Road_number,longitude,latitude,geometry
604,176137,LCM-LCM19200834-IM_1,accident,2019-11-06 17:33:14,2019-11-06 18:49:06,A200,4.695898,52.384174,POINT (52.38417 4.69590)
2160,179250,LCM-LCM19206007-IM_1,vehicle_obstruction,2019-11-14 17:04:51,2019-11-14 18:20:31,A200,4.704684,52.384193,POINT (52.38419 4.70468)
3667,1637,NLRWS_NLSIT002608644_1,general_obstruction,2019-08-19 17:21:00,2019-08-19 18:01:21,A200,4.668809,52.382328,POINT (52.38233 4.66881)
3792,2064,NLRWS_NLSIT002610155_1,general_obstruction,2019-08-23 12:18:00,2019-08-23 12:39:04,A200,4.694430,52.384090,POINT (52.38409 4.69443)
4480,4534,NLRWS_NLSIT002626921_1,general_obstruction,2019-09-20 12:17:00,2019-09-20 12:18:48,A200,4.695898,52.384174,POINT (52.38417 4.69590)


In [56]:
gdf_incident_a200['geometry'].iloc[0].xy[0][0]

len(gdf_incident_a200.geometry)

70

In [104]:
from folium import plugins
map = folium.Map(location=[52.399190, 4.893658])
gjson = folium.features.GeoJson(
    a200_network,
).add_to(map)

geo_df_list = [[gdf_incident_a200['geometry'].iloc[i].xy[0][0],
                gdf_incident_a200['geometry'].iloc[i].xy[1][0]] for i in range(len(gdf_incident_a200.geometry))]

incident_a200_mark = plugins.HeatMap(geo_df_list).add_to(map)

map

In [79]:
gdf_incident_a200.geometry.iloc[0].distance(df_a200.geometry.iloc[1])

501251.7237635516

In [106]:
dist = []
gdf_incident_a200 = gdf_incident_a200.to_crs('EPSG:4326')
df_a200 = df_a200.to_crs('EPSG:4326')
# for index, row in df_a200.iterrows():
#     if gdf_incident_a200.geometry.iloc[0].distance(row.geometry) < 1000000:
#         print(f"the road section id is: {row['Road_section_id']}")
#         break
#     else:
#         continue

for index, row in df_a200.iterrows():
    dis = gdf_incident_a200.geometry.iloc[25].distance(row.geometry)
    dist.append(dis)

In [107]:
print(min(dist))

499889.5725455427


In [91]:
data

{'geometry': [<LINESTRING (0 0, 2 2, 4 4)>, <POINT (1 1)>]}